In [1]:
import tensorflow as tf
import cv2, numpy as np, os
import pandas as pd, time
import tensorflow.contrib as tf_contrib

DECAY_BN = 0.0001
EPSILON_BN = 1e-05

C:\Users\sumanth\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def read_images(path, ran, phase):
    save_path = phase+"_images.npy"
    if(os.path.exists(save_path)):
        print("npy file available...")
        return np.load(save_path)
    images = []
    for r, d, f in os.walk(path):
        for i in range(1, ran+1):
            img_name = os.path.join(r, str(i)+".jpg")
            images.append(cv2.imread(img_name)/255.)
    print("done loading from " + path)
    images = np.array(images)
    np.save(save_path, images)
    return images

def read_labels(csv_file, phase, nb_classes = 8):
    save_path = phase+"_labels.npy"
    labels = np.array(pd.read_csv(csv_file, header = None).values)
    labels = np.reshape(labels, (labels.shape[1], labels.shape[0]))
    one_hot_targets = np.eye(nb_classes)[[i-1 for i in labels]]
    one_hot_targets = np.reshape(one_hot_targets, (one_hot_targets.shape[0], 8))
    np.save(save_path, one_hot_targets)
    return one_hot_targets
    

def get_num_trainable_params():
    total_parameters = 0
    for variable in tf.trainable_variables():
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    print(total_parameters)

In [3]:
train_images = read_images("./train/", 1888, "train")
test_images = read_images("./test/", 800, "test")

train_labels = read_labels("./train_labels.csv", "train")
test_labels = read_labels("./test_labels.csv", "test")

print("Train images: " + str(train_images.shape))
print("Test images:  " + str(test_images.shape))

print("Train labels: " + str(train_labels.shape))
print("Test labels:  " + str(test_labels.shape))

npy file available...
npy file available...
Train images: (1888, 256, 256, 3)
Test images:  (800, 256, 256, 3)
Train labels: (1888, 8)
Test labels:  (800, 8)


In [30]:
def residual_block(x, filter_size, wt_init, weight_regularizer, conv_number, instance, is_train, stride = 1, reuse = False):
    prev = x
    
    
        
    x = tf.layers.conv2d(x, filters=filter_size, kernel_size=3, kernel_initializer=wt_init, strides=stride,
                                kernel_regularizer=weight_regularizer, padding="SAME", trainable=is_train, name="shortcut_conv"+conv_number+"_"+instance, reuse=reuse)
    x = tf_contrib.layers.batch_norm(x, decay=DECAY_BN, epsilon=EPSILON_BN, center=True, scale=True, trainable=is_train, reuse=reuse, scope="short_batch_norm"+conv_number+"_"+instance)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x, filters=filter_size, kernel_size=3, kernel_initializer=wt_init, trainable=is_train,
                         kernel_regularizer=weight_regularizer, name="conv"+conv_number+"_"+instance, padding="SAME", reuse=reuse)
    
    x = tf_contrib.layers.batch_norm(x, decay=DECAY_BN, epsilon=EPSILON_BN, center=True, scale=True, trainable=is_train, reuse=reuse, scope="batch_norm"+conv_number+"_"+instance)
    
    if(stride == 2):
        #x = prev + x
        prev = tf.layers.conv2d(prev, filters=filter_size, kernel_size=1, kernel_initializer=wt_init, strides=stride,
                                kernel_regularizer=weight_regularizer, padding="SAME", trainable=is_train, name="1x1_"+conv_number+"_"+instance, reuse=reuse)
    
    x = prev + x
    x = tf.nn.relu(x)
    
    return x

def build_model(train_input, is_train = False, reuse = False):
    wt_init = tf_contrib.layers.variance_scaling_initializer()
    weight_regularizer = tf_contrib.layers.l2_regularizer(0.0001)
    
    x = tf.layers.conv2d(train_input, filters=64, kernel_size=7, kernel_initializer=wt_init, strides=2, 
                         kernel_regularizer=weight_regularizer, trainable=is_train, name="conv1", padding='SAME', reuse=reuse)
    x = tf_contrib.layers.batch_norm(x, decay=DECAY_BN, epsilon=EPSILON_BN, center=True, scale=True, trainable=is_train, reuse=reuse, scope="batch_norm1")
    x = tf.nn.relu(x)
    
    x = tf.layers.max_pooling2d(x, pool_size = 2, strides = 2, name="maxpool") # 56x56x64
    
    x = residual_block(x, 64, wt_init, weight_regularizer, "2", "1", is_train, reuse=reuse)
    x = residual_block(x, 64, wt_init, weight_regularizer, "2", "2", is_train, reuse=reuse)
    
    x = residual_block(x, 128, wt_init, weight_regularizer, "3", "1", is_train, stride=2, reuse=reuse)
    x = residual_block(x, 128, wt_init, weight_regularizer, "3", "2", is_train, reuse=reuse)
    
    x = residual_block(x, 256, wt_init, weight_regularizer, "4", "1", is_train, stride=2, reuse=reuse)
    x = residual_block(x, 256, wt_init, weight_regularizer, "4", "2", is_train, reuse=reuse)
    
    x = residual_block(x, 512, wt_init, weight_regularizer, "5", "1", is_train, stride=2, reuse=reuse)
    x = residual_block(x, 512, wt_init, weight_regularizer, "5", "2", is_train, reuse=reuse)
    
    x = tf.layers.average_pooling2d(x, pool_size=7, strides=2, name="avgpool")
    x = tf.contrib.layers.flatten(x)
    
    #x = tf.layers.dense(x, 1000, activation=tf.nn.relu, kernel_initializer=wt_init, trainable=is_train, name="fc1")
    x = tf.layers.dense(x, 8, kernel_initializer=wt_init, trainable=is_train, name="output", reuse=reuse)
    #x = tf.nn.softmax(x, name="output_logits")
    
    return x

def get_losses(output, label):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=label))
    prediction = tf.equal(tf.argmax(output, -1), tf.argmax(label, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    
    return loss, accuracy

In [31]:
tf.reset_default_graph()
train_input = tf.placeholder(tf.float32, (None, 224, 224, 3), name = "train_input")
train_label = tf.placeholder(tf.int32, (None, 8), name = "train_label")
test_input = tf.placeholder(tf.float32, (None, 224, 224, 3), name = "test_input")
test_label = tf.placeholder(tf.int32, (None, 8), name = "test_label")

batch = 32

lr = tf.placeholder(tf.float32, name='learning_rate')
train_output = build_model(train_input, is_train = True)
#test_output = build_model(test_input, reuse = True)

train_loss, train_acc = get_losses(train_output, train_label)
#test_loss, test_acc = get_losses(test_output, test_label)

optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9).minimize(train_loss)

#### Summary #####
summary_train_loss = tf.summary.scalar("train_loss", train_loss)
#summary_test_loss = tf.summary.scalar("train_loss", test_loss)

summary_train_accuracy = tf.summary.scalar("train_accuracy", train_acc)
#summary_test_accuracy = tf.summary.scalar("train_accuracy", test_acc)

train_summary = tf.summary.merge([summary_train_loss, summary_train_accuracy])
#test_summary = tf.summary.merge([summary_test_loss, summary_test_accuracy])

#get_num_trainable_params()

In [32]:
def get_mini_batches(X, y, crop = True, batch_size = 32):
    
    n_minibatches = X.shape[0] // batch_size
    data = list(zip(X, y))
    np.random.shuffle(data)
    X, y = zip(*data)
    X = np.array(X)
    y = np.array(y)
    mini_batches = []
    i = 0
    for i in range(n_minibatches):
        X_mini = X[i * batch_size:(i + 1)*batch_size]
        if(crop):
            X_mini = data_augmentation(X_mini, 224)
        y_mini = y[i * batch_size:(i + 1)*batch_size]
        mini_batches.append((X_mini, y_mini))
    if(X.shape[0] % batch_size != 0):
        X_mini = X[i * batch_size:X.shape[0]]
        if(crop):
            X_mini = data_augmentation(X_mini, 224)
        y_mini = y[i * batch_size:y.shape[0]]
        mini_batches.append((X_mini, y_mini))
    return mini_batches, n_minibatches

def _random_crop(batch, crop_shape, padding=None):
    
    oshape = np.shape(batch[0])
    if padding:
        oshape = (oshape[0] + 2 * padding, oshape[1] + 2 * padding)
    new_batch = []
    npad = ((padding, padding), (padding, padding), (0, 0))
    for i in range(len(batch)):
        new_batch.append(batch[i])
        if padding:
            new_batch[i] = np.lib.pad(batch[i], pad_width=npad, mode='constant', constant_values=0)
        nh = np.random.randint(0, oshape[0] - crop_shape[0])
        nw = np.random.randint(0, oshape[1] - crop_shape[1])
        new_batch[i] = new_batch[i][nh:nh + crop_shape[0], nw:nw + crop_shape[1]]
    return new_batch

def data_augmentation(batch, img_size):
     return _random_crop(batch, [img_size, img_size, 3])

In [33]:
epochs = 30
epoch_lr = 0.1
counter = 0
test_counter = 0

sess = tf.Session()
sess.run(tf.global_variables_initializer())

checkpoint_dir = "./checkpoints"
writer = tf.summary.FileWriter("./logs", sess.graph)
saver = tf.train.Saver()

#test_feed_dict = {test_input: train_images, test_label: train_labels}

start_time = time.time()
for epoch in range(epochs):
    if epoch == int(epochs * 0.5) or epoch == int(epochs * 0.75):
        epoch_lr = epoch_lr * 0.1
    minibatches, n_batches = get_mini_batches(train_images, train_labels, batch_size = batch)
    i = 0
    for minibatch in minibatches:
        i += 1
        X, y = minibatch
        
        train_feed_dict = {train_input:X, train_label: y, lr: epoch_lr}
        
        acc, _, l, summary1 = sess.run([train_acc, optimizer, train_loss, train_summary], feed_dict=train_feed_dict)
        writer.add_summary(summary1, counter)
        counter += 1
        
        print("Epoch: [%2d] [%5d/%5d] time: %4.4f, train_accuracy: %.2f, learning_rate : %.4f, train_loss: %.2f" \
                      % (epoch, i, n_batches, time.time() - start_time, acc, epoch_lr, l))
    if(epoch%5 == 0):
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver.save(sess, os.path.join(checkpoint_dir, 'ResNet18.model'), global_step=counter)

    
sess.close()

Epoch: [ 0] [    1/   59] time: 2.3751, train_accuracy: 0.19, learning_rate : 0.1000, train_loss: 4.38
Epoch: [ 0] [    2/   59] time: 2.8750, train_accuracy: 0.09, learning_rate : 0.1000, train_loss: 41.70
Epoch: [ 0] [    3/   59] time: 3.3914, train_accuracy: 0.16, learning_rate : 0.1000, train_loss: 20.92
Epoch: [ 0] [    4/   59] time: 3.9069, train_accuracy: 0.09, learning_rate : 0.1000, train_loss: 19.33
Epoch: [ 0] [    5/   59] time: 4.4227, train_accuracy: 0.25, learning_rate : 0.1000, train_loss: 16.83
Epoch: [ 0] [    6/   59] time: 4.9383, train_accuracy: 0.12, learning_rate : 0.1000, train_loss: 9.55
Epoch: [ 0] [    7/   59] time: 5.4540, train_accuracy: 0.16, learning_rate : 0.1000, train_loss: 14.37
Epoch: [ 0] [    8/   59] time: 5.9539, train_accuracy: 0.12, learning_rate : 0.1000, train_loss: 8.67
Epoch: [ 0] [    9/   59] time: 6.4697, train_accuracy: 0.09, learning_rate : 0.1000, train_loss: 7.74
Epoch: [ 0] [   10/   59] time: 6.9696, train_accuracy: 0.03, learni

Epoch: [ 1] [   21/   59] time: 46.6617, train_accuracy: 0.31, learning_rate : 0.1000, train_loss: 1.93
Epoch: [ 1] [   22/   59] time: 47.1618, train_accuracy: 0.16, learning_rate : 0.1000, train_loss: 2.22
Epoch: [ 1] [   23/   59] time: 47.6774, train_accuracy: 0.28, learning_rate : 0.1000, train_loss: 2.05
Epoch: [ 1] [   24/   59] time: 48.1931, train_accuracy: 0.03, learning_rate : 0.1000, train_loss: 2.34
Epoch: [ 1] [   25/   59] time: 48.7087, train_accuracy: 0.16, learning_rate : 0.1000, train_loss: 1.97
Epoch: [ 1] [   26/   59] time: 49.2244, train_accuracy: 0.16, learning_rate : 0.1000, train_loss: 2.21
Epoch: [ 1] [   27/   59] time: 49.7244, train_accuracy: 0.19, learning_rate : 0.1000, train_loss: 2.16
Epoch: [ 1] [   28/   59] time: 50.2401, train_accuracy: 0.12, learning_rate : 0.1000, train_loss: 2.17
Epoch: [ 1] [   29/   59] time: 50.7556, train_accuracy: 0.12, learning_rate : 0.1000, train_loss: 2.04
Epoch: [ 1] [   30/   59] time: 51.2558, train_accuracy: 0.12, l

Epoch: [ 2] [   41/   59] time: 88.8827, train_accuracy: 0.06, learning_rate : 0.1000, train_loss: 2.14
Epoch: [ 2] [   42/   59] time: 89.3985, train_accuracy: 0.22, learning_rate : 0.1000, train_loss: 1.89
Epoch: [ 2] [   43/   59] time: 89.9140, train_accuracy: 0.12, learning_rate : 0.1000, train_loss: 2.18
Epoch: [ 2] [   44/   59] time: 90.4142, train_accuracy: 0.28, learning_rate : 0.1000, train_loss: 1.88
Epoch: [ 2] [   45/   59] time: 90.9297, train_accuracy: 0.16, learning_rate : 0.1000, train_loss: 2.12
Epoch: [ 2] [   46/   59] time: 91.4455, train_accuracy: 0.19, learning_rate : 0.1000, train_loss: 2.12
Epoch: [ 2] [   47/   59] time: 91.9610, train_accuracy: 0.16, learning_rate : 0.1000, train_loss: 1.92
Epoch: [ 2] [   48/   59] time: 92.4612, train_accuracy: 0.06, learning_rate : 0.1000, train_loss: 2.18
Epoch: [ 2] [   49/   59] time: 92.9923, train_accuracy: 0.22, learning_rate : 0.1000, train_loss: 1.81
Epoch: [ 2] [   50/   59] time: 93.4924, train_accuracy: 0.28, l

Epoch: [ 4] [    2/   59] time: 133.4808, train_accuracy: 0.25, learning_rate : 0.1000, train_loss: 2.03
Epoch: [ 4] [    3/   59] time: 133.9963, train_accuracy: 0.25, learning_rate : 0.1000, train_loss: 1.80
Epoch: [ 4] [    4/   59] time: 134.5277, train_accuracy: 0.31, learning_rate : 0.1000, train_loss: 1.69
Epoch: [ 4] [    5/   59] time: 135.0433, train_accuracy: 0.25, learning_rate : 0.1000, train_loss: 2.05
Epoch: [ 4] [    6/   59] time: 135.5747, train_accuracy: 0.56, learning_rate : 0.1000, train_loss: 1.95
Epoch: [ 4] [    7/   59] time: 136.1058, train_accuracy: 0.25, learning_rate : 0.1000, train_loss: 1.75
Epoch: [ 4] [    8/   59] time: 136.6372, train_accuracy: 0.31, learning_rate : 0.1000, train_loss: 2.27
Epoch: [ 4] [    9/   59] time: 137.1530, train_accuracy: 0.28, learning_rate : 0.1000, train_loss: 1.77
Epoch: [ 4] [   10/   59] time: 137.6998, train_accuracy: 0.31, learning_rate : 0.1000, train_loss: 1.95
Epoch: [ 4] [   11/   59] time: 138.2156, train_accurac

Epoch: [ 5] [   22/   59] time: 175.8607, train_accuracy: 0.44, learning_rate : 0.1000, train_loss: 1.47
Epoch: [ 5] [   23/   59] time: 176.3918, train_accuracy: 0.53, learning_rate : 0.1000, train_loss: 1.40
Epoch: [ 5] [   24/   59] time: 176.9073, train_accuracy: 0.50, learning_rate : 0.1000, train_loss: 1.41
Epoch: [ 5] [   25/   59] time: 177.4249, train_accuracy: 0.47, learning_rate : 0.1000, train_loss: 1.31
Epoch: [ 5] [   26/   59] time: 177.9561, train_accuracy: 0.34, learning_rate : 0.1000, train_loss: 1.56
Epoch: [ 5] [   27/   59] time: 178.4875, train_accuracy: 0.34, learning_rate : 0.1000, train_loss: 1.65
Epoch: [ 5] [   28/   59] time: 179.0030, train_accuracy: 0.44, learning_rate : 0.1000, train_loss: 2.05
Epoch: [ 5] [   29/   59] time: 179.5188, train_accuracy: 0.31, learning_rate : 0.1000, train_loss: 1.65
Epoch: [ 5] [   30/   59] time: 180.0499, train_accuracy: 0.38, learning_rate : 0.1000, train_loss: 2.00
Epoch: [ 5] [   31/   59] time: 180.5657, train_accurac

...

Epoch: [43] [   59/   59] time: 1511.2284, train_accuracy: 0.94, learning_rate : 0.0010, train_loss: 0.21
Epoch: [44] [    1/   59] time: 1515.4651, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.29
Epoch: [44] [    2/   59] time: 1515.9986, train_accuracy: 0.84, learning_rate : 0.0010, train_loss: 0.54
Epoch: [44] [    3/   59] time: 1516.5172, train_accuracy: 0.97, learning_rate : 0.0010, train_loss: 0.13
Epoch: [44] [    4/   59] time: 1517.0468, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.33
Epoch: [44] [    5/   59] time: 1517.5804, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.33
Epoch: [44] [    6/   59] time: 1518.1010, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.27
Epoch: [44] [    7/   59] time: 1518.6027, train_accuracy: 0.94, learning_rate : 0.0010, train_loss: 0.27
Epoch: [44] [    8/   59] time: 1519.1113, train_accuracy: 0.94, learning_rate : 0.0010, train_loss: 0.18
Epoch: [44] [    9/   59] time: 1519.6259, tra

Epoch: [45] [   19/   59] time: 1559.6499, train_accuracy: 0.78, learning_rate : 0.0010, train_loss: 0.45
Epoch: [45] [   20/   59] time: 1560.1785, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.24
Epoch: [45] [   21/   59] time: 1560.7041, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.31
Epoch: [45] [   22/   59] time: 1561.2347, train_accuracy: 0.91, learning_rate : 0.0010, train_loss: 0.36
Epoch: [45] [   23/   59] time: 1561.7623, train_accuracy: 0.91, learning_rate : 0.0010, train_loss: 0.27
Epoch: [45] [   24/   59] time: 1562.2948, train_accuracy: 0.91, learning_rate : 0.0010, train_loss: 0.35
Epoch: [45] [   25/   59] time: 1562.8174, train_accuracy: 0.91, learning_rate : 0.0010, train_loss: 0.19
Epoch: [45] [   26/   59] time: 1563.3291, train_accuracy: 0.94, learning_rate : 0.0010, train_loss: 0.15
Epoch: [45] [   27/   59] time: 1563.8567, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.29
Epoch: [45] [   28/   59] time: 1564.3922, tra

Epoch: [46] [   38/   59] time: 1606.1967, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.35
Epoch: [46] [   39/   59] time: 1606.7292, train_accuracy: 0.97, learning_rate : 0.0010, train_loss: 0.19
Epoch: [46] [   40/   59] time: 1607.2678, train_accuracy: 0.94, learning_rate : 0.0010, train_loss: 0.19
Epoch: [46] [   41/   59] time: 1607.8004, train_accuracy: 0.91, learning_rate : 0.0010, train_loss: 0.21
Epoch: [46] [   42/   59] time: 1608.3349, train_accuracy: 1.00, learning_rate : 0.0010, train_loss: 0.12
Epoch: [46] [   43/   59] time: 1608.8625, train_accuracy: 0.97, learning_rate : 0.0010, train_loss: 0.20
Epoch: [46] [   44/   59] time: 1609.3951, train_accuracy: 0.94, learning_rate : 0.0010, train_loss: 0.26
Epoch: [46] [   45/   59] time: 1609.9147, train_accuracy: 0.94, learning_rate : 0.0010, train_loss: 0.15
Epoch: [46] [   46/   59] time: 1610.4313, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.42
Epoch: [46] [   47/   59] time: 1610.9659, tra

Epoch: [47] [   57/   59] time: 1651.4078, train_accuracy: 0.81, learning_rate : 0.0010, train_loss: 0.48
Epoch: [47] [   58/   59] time: 1651.9254, train_accuracy: 0.84, learning_rate : 0.0010, train_loss: 0.31
Epoch: [47] [   59/   59] time: 1652.4560, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.30
Epoch: [48] [    1/   59] time: 1656.6926, train_accuracy: 0.91, learning_rate : 0.0010, train_loss: 0.24
Epoch: [48] [    2/   59] time: 1657.2222, train_accuracy: 0.91, learning_rate : 0.0010, train_loss: 0.35
Epoch: [48] [    3/   59] time: 1657.7388, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.46
Epoch: [48] [    4/   59] time: 1658.2734, train_accuracy: 0.81, learning_rate : 0.0010, train_loss: 0.27
Epoch: [48] [    5/   59] time: 1658.8100, train_accuracy: 0.81, learning_rate : 0.0010, train_loss: 0.41
Epoch: [48] [    6/   59] time: 1659.3396, train_accuracy: 0.94, learning_rate : 0.0010, train_loss: 0.22
Epoch: [48] [    7/   59] time: 1659.8721, tra

Epoch: [49] [   17/   59] time: 1699.9968, train_accuracy: 0.97, learning_rate : 0.0010, train_loss: 0.21
Epoch: [49] [   18/   59] time: 1700.5274, train_accuracy: 0.94, learning_rate : 0.0010, train_loss: 0.42
Epoch: [49] [   19/   59] time: 1701.0580, train_accuracy: 0.94, learning_rate : 0.0010, train_loss: 0.26
Epoch: [49] [   20/   59] time: 1701.5916, train_accuracy: 0.84, learning_rate : 0.0010, train_loss: 0.30
Epoch: [49] [   21/   59] time: 1702.1172, train_accuracy: 0.81, learning_rate : 0.0010, train_loss: 0.65
Epoch: [49] [   22/   59] time: 1702.6567, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.44
Epoch: [49] [   23/   59] time: 1703.1833, train_accuracy: 0.84, learning_rate : 0.0010, train_loss: 0.44
Epoch: [49] [   24/   59] time: 1703.7119, train_accuracy: 0.81, learning_rate : 0.0010, train_loss: 0.41
Epoch: [49] [   25/   59] time: 1704.2315, train_accuracy: 0.88, learning_rate : 0.0010, train_loss: 0.32
Epoch: [49] [   26/   59] time: 1704.7531, tra

In [ ]:
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

# out = sess.run(train_output, feed_dict={train_input: data_augmentation(test_images[0:15], 224)})

# print(out)
# sess.close()

In [ ]:
# mini, n = get_mini_batches(train_images[0:15], train_labels[0:15])
# for i in mini:
#     x, y = i
#     break
    
# x = np.array(x)
# y = np.array(y)

# idx = 2
# import matplotlib.pyplot as plt
# plt.imshow(x[idx])
# y[idx]